# Logging

In [ ]:
import os
import sys
import logging
import random

## Sources

- [Logging in python: developer guide (rus)](https://habr.com/ru/companies/wunderfund/articles/683880/);
- ["Logging HOWTO"](https://docs.python.org/3/howto/logging.html#configuring-logging) from docs.python.org contains two tutorials: basic and advanced.

## Error types

There are five levels of logging:

- Debug;
- Info;
- Warning;
- Error;
- Critical.

You can call them by using the relevant functions:

In [2]:
logging.debug("A DEBUG Message")
logging.info("An INFO")
logging.warning("A WARNING")
logging.error("An ERROR")
logging.critical("A message of CRITICAL severity")

ERROR:root:An ERROR
CRITICAL:root:A message of CRITICAL severity


In previous examle `debug` and `info` errors wasn't printed. It happened becaulse default value for `logging.level` setted to `logging.WARNING`, therefore only those messages above `WARNING` will be displayed. 

## Basic logger

This logger is used when you run the logging methods of the `logging` module itself.

Using the `logging.basicConfig` function, you can define the behaviour of the logs generated by the `logging` module. The following example defines the `DEBUG` logging level so that you can later see any message raised by the `logging` methods. By default only `warning`, `error` and `critical` are executed.

**Note** In the following example, the code is saving the logs to a separate file and executing it from another interpreter. This approach is necessary because once you execute any logging method in this notebook, you cannot modify its configuration anymore.

In [3]:
%%writefile logging_files/basic_logger.py
import logging
logging.basicConfig(level=logging.DEBUG)

logging.debug("A DEBUG Message")
logging.info("An INFO")
logging.warning("A WARNING")
logging.error("An ERROR")
logging.critical("A message of CRITICAL severity")

Overwriting logging_files/basic_logger.py


In [4]:
!python3 logging_files/basic_logger.py

DEBUG:root:A DEBUG Message
INFO:root:An INFO
ERROR:root:An ERROR
CRITICAL:root:A message of CRITICAL severity


### Format

To set up the format for the basic logger, you should use the `format` parameter of the `logging.basicConfig` function.

The following example sets up basic logger to print the time of the event and it's message.

In [4]:
%%writefile logging_files/basic_logger.py
import logging
logging.basicConfig(format='%(asctime)s %(message)s')

logging.debug("A DEBUG Message")
logging.info("An INFO")
logging.warning("A WARNING")
logging.error("An ERROR")
logging.critical("A message of CRITICAL severity")

Overwriting logging_files/basic_logger.py


In [5]:
!python3 logging_files/basic_logger.py

2024-05-06 15:43:39,687 A WARNING
2024-05-06 15:43:39,687 An ERROR
2024-05-06 15:43:39,687 A message of CRITICAL severity


To check format of the basic logger you need:

- Use `logging.getLogger()` to get the basic logger;
- Grab the handler you're interested in. By default, basic loggers have only one handler;
- The `formatter._fmt` field of the handler would be a string defining the format.

The following cell prints the notebooks handler format.

In [23]:
# Retrieve the root logger
root_logger = logging.getLogger()
# Retrieve the handlers configured for the root logger
handler = root_logger.handlers[0]
formatter = handler.formatter
formatter._fmt

'%(levelname)s:%(name)s:%(message)s'

## Common structure

In real work, basic loggers are rarely used. Usually such objects are created as

- Loggers;
- Handlers;
- Formattres;
- Filtres.

Each defines some aspect of log generation.

### Loggers

For different parts of the programs you'll need to have defferent `loggers`. So you can create new loger by using `logging.getLogger(<loger name>)`.

In [5]:
logger1 = logging.getLogger("logger1")
logger1.setLevel(logging.INFO)
print("=====logger1=====", file=sys.stderr)
logger1.debug("A DEBUG Message")
logger1.info("An INFO")
logger1.warning("A WARNING")
logger1.error("An ERROR")
logger1.critical("A message of CRITICAL severity")


logger2 = logging.getLogger("logger2")
logger2.setLevel(logging.ERROR)
print("=====logger2=====", file=sys.stderr)
logger2.debug("A DEBUG Message")
logger2.info("An INFO")
logger2.warning("A WARNING")
logger2.error("An ERROR")
logger2.critical("A message of CRITICAL severity")

=====logger1=====
INFO:logger1:An INFO
ERROR:logger1:An ERROR
CRITICAL:logger1:A message of CRITICAL severity
=====logger2=====
ERROR:logger2:An ERROR
CRITICAL:logger2:A message of CRITICAL severity


### Handlers

Defines the direction in which logs are written. It can be files or output streams. You can add handlers to the logger using the `logger.addHandler` method.

Check list of the [usefull handlers](https://docs.python.org/3/howto/logging.html#useful-handlers).

In [3]:
logger = logging.getLogger("temp_logger")
stream_handler = logging.StreamHandler()
logger.addHandler(stream_handler)
logger.error("It's a error!")
# to prevent adding a handler for each
# run of this cell, we'll delete just
# added logger
del logger.handlers[0]

It's a error!


### Formatters

Formatters define how each line of the log is to be printed. You must set the formatter for the handler using the `handler.setFormatter` method.

In [5]:
logger = logging.getLogger("some_logger")
stream_handler = logging.StreamHandler()
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)
logger.error("It's error")
del logger.handlers[0]

2024-05-07 10:42:59,505 - some_logger - ERROR - It's error


## Configuration

There are some tools that allows to configure `logging` module from optional `key:value` like formats.

There is [special page](logging/configuration.ipynb) about aspects of working with it.

Here is a simple example of how to configure a whole logging module just by using Python dictionaries.

Here is a logger created with nanme `simpleExample` with a really specific formatter - so the logs have the appropriate shape.

In [19]:
logging.config.dictConfig({
    "version" : 1,
    "formatters" : {
        "simpleFormatter" : {
            "format" : "I'm created from config :) %(message)s"
        }
    },
    "handlers" : {
        "consoleHandler" : {
            "class" : "logging.StreamHandler",
            "formatter" : "simpleFormatter"
        }
    },
    "loggers" : {
        "simpleExample" : {
            "handlers" : ["consoleHandler"]
        }
    }
})

logging.getLogger("simpleExample").error("{it's message}")

I'm created from config :) {it's message}


It is the preferred way to configure the logs, because anyone who is not involved with the code can participate in configuring the logs.

## List all loggers

It's usefull to acesss all loggers available now. All loogets defined in the cureent interprier state is defined in the `logging.Logger.manager.loggerDict`.

The following cell demonstrates how `loggerDict` looks like for the current jupyter notebook.

In [3]:
logging.Logger.manager.loggerDict

{'concurrent.futures': <Logger concurrent.futures (WARNING)>,
 'concurrent': <Logger concurrent (WARNING)>,
 'asyncio': <Logger asyncio (WARNING)>,
 'tornado.access': <Logger tornado.access (WARNING)>,
 'tornado': <Logger tornado (WARNING)>,
 'tornado.application': <Logger tornado.application (WARNING)>,
 'tornado.general': <Logger tornado.general (WARNING)>,
 'stack_data.serializing': <Logger stack_data.serializing (WARNING)>,
 'stack_data': <Logger stack_data (WARNING)>,
 'parso': <Logger parso (WARNING)>,
 'prompt_toolkit.buffer': <Logger prompt_toolkit.buffer (WARNING)>,
 'prompt_toolkit': <Logger prompt_toolkit (WARNING)>,
 'parso.python.diff': <Logger parso.python.diff (WARNING)>,
 'parso.python': <Logger parso.python (WARNING)>,
 'parso.cache': <Logger parso.cache (WARNING)>,
 'Comm': <Logger Comm (WARNING)>,
 'ipykernel.comm': <Logger ipykernel.comm (WARNING)>,
 'ipykernel': <Logger ipykernel (WARNING)>,
 'IPKernelApp': <Logger IPKernelApp (DEBUG)>}